

``` postgresql
BEGIN;

create temporary table t 
(
  Duration NUMERIC,
  Start_time TIMESTAMP,
  End_time TIMESTAMP,
  Start_Station NUMERIC,
  Start_Station_Name VARCHAR,
  End_Station NUMERIC,
  End_Station_Name VARCHAR,
  Bike VARCHAR,
  Member VARCHAR 
);

COPY t 
FROM '/tmp/2013Q4-capitalbikeshare-tripdata.csv' DELIMITER ',' CSV HEADER;

INSERT INTO rides(Duration, Start, Bike, Member)
SELECT Duration, Start_time, Bike, Member
FROM t;

drop table t;

SELECT DATE_TRUNC('month', start) AS Month, COUNT(bike) AS Num_Rides  
FROM rides
GROUP BY Month;

COMMIT;

```


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing

In [ ]:
bike = pd.read_csv('./data/rides_monthly_aggregate.csv', usecols = [1])
bike.set_index(pd.period_range('1/1/2013', freq='M', periods=84), inplace = True)

bike.head()

In [ ]:
bike.plot(figsize = (10,6))

In [ ]:
fit_hw = ExponentialSmoothing(bike, seasonal_periods = 12, trend = 'add', seasonal = 'add').fit()

In [ ]:
bike_plot = bike.plot(figsize = (10,6), title = "Holt-Winters' Method Bike Share Fit")
bike_plot.set_ylabel("Number of Bike rentals")
bike_plot.set_xlabel("Year")

fit_hw.fittedvalues.plot(ax = bike_plot, style = '--', color = 'DarkRed')
bike_plot.legend(['Bike Rentals', 'H-W Model Fit'])

In [ ]:
states_decomp = pd.DataFrame(np.c_[fit_hw.level, fit_hw.slope, fit_hw.season], columns=['level','slope','seasonal'], index=bike.index)

fig, [ax1, ax2, ax3] = plt.subplots(3, 1, figsize=(10,6))
states_decomp[['level']].plot(ax = ax1, title = "Level")
states_decomp[['slope']].plot(ax = ax2, title = "Trend/Slope")
states_decomp[['seasonal']].plot(ax = ax3, title = "Seasonality")